In [1]:
from architectures_v1 import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, RNN, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import time
import random

## Loading Training/Testing/Validation Data

In [3]:
dataset = "PhonemeSpectra"

x_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_train.npz')))
x_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'x_test.npz')))

x_train = x_train_load['arr_0']
x_test = x_test_load['arr_0']

x_all = np.concatenate((x_train, x_test), axis = 0)

n_instances = x_all.shape[0]
print(f"Total Instances: {n_instances}")

y_train_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_train.npz')))
y_test_load = np.load(os.path.abspath(os.path.join('99_data', dataset,'y_test.npz')))

y_train = np.asarray(y_train_load['arr_0'], dtype = np.uint64)
y_test = np.asarray(y_test_load['arr_0'], dtype = np.uint64)

y_all = np.concatenate((y_train, y_test), axis = 0)

n_validation = int(0.1*n_instances)
print(f"Validation Instances: {n_validation}")

ind_validation = random.sample(range(0, n_instances), n_validation)
x_valid = x_all[ind_validation, :, :]
y_valid = y_all[ind_validation]

x_all = np.delete(x_all, ind_validation, axis = 0)
y_all = np.delete(y_all, ind_validation, axis = 0)

print(f"x_all: {x_all.shape} - y_all: {y_all.shape}")
print(f"x_valid: {x_valid.shape} - y_valid: {y_valid.shape}")

Total Instances: 6668
Validation Instances: 666
x_all: (6002, 217, 11) - y_all: (6002,)
x_valid: (666, 217, 11) - y_valid: (666,)


In [4]:
seed = 0
tf.random.set_random_seed(seed)
np.random.seed(seed)

length = x_all.shape[1]
n_features = x_all.shape[-1]

model = Sequential()

model.add(LSTM(units = 212, 
              input_shape = (length, n_features), return_sequences = False))
model.add(Dense(to_categorical(y_all).shape[-1], activation = "softmax"))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 212)               189952    
_________________________________________________________________
dense (Dense)                (None, 39)                8307      
Total params: 198,259
Trainable params: 198,259
Non-trainable params: 0
_________________________________________________________________


In [5]:
if not os.path.exists(os.path.abspath(os.path.join('models', dataset))):
    os.mkdir(os.path.abspath(os.path.join('models', dataset)))

kf = KFold(n_splits = 5)
sdk = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
n_fold = 1

acc_per_fold = []
epoch_per_fold = []
loss_per_fold = []
rec_per_fold = []
prec_per_fold = []
f1_per_fold = []

for train, test in sdk.split(x_all, y_all):
    
    file_path = os.path.abspath(os.path.join('models', dataset, f'{dataset}-LSTM_{n_fold}.hdf5'))

    callbacks = [
        ModelCheckpoint(filepath=file_path, monitor='val_loss', save_best_only = True, mode = 'min', verbose = 1), 
        EarlyStopping(monitor = 'val_loss', patience = 50, mode = 'min')]

    t = time.time()
    
    print(f"Fold: {n_fold}\n")
    result = model.fit(x_all[train], 
                       to_categorical(y_all[train]),
                       epochs = 2000, 
                       batch_size = 100, 
                       validation_data = (x_valid, to_categorical(y_valid)), 
                       callbacks = callbacks)

    print(f"Training time: {time.time() - t} s")

    df_results = pd.DataFrame(result.history)
    df_results.to_csv(os.path.abspath(os.path.join('models', dataset, f'LSTM_results_{n_fold}.csv')))
    
    model.load_weights(file_path)
    scores = model.evaluate(x_all[test], to_categorical(y_all[test]))
    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1])
    epoch_per_fold.append(np.argmin(result.history['val_loss']))
    
    # Computing predictions
    y_pred_test = np.argmax(model.predict(x_all[test]), axis = 1)
    
    f1_per_fold.append(f1_score(y_all[test], y_pred_test, average = 'macro'))
    rec_per_fold.append(recall_score(y_all[test], y_pred_test, average = 'macro'))
    prec_per_fold.append(precision_score(y_all[test], y_pred_test, average = 'macro'))
    
    n_fold += 1
    
df_scores = pd.DataFrame(columns = ['F1', 'Loss', 'Accuracy', 'Precision', 'Recall'])
df_scores['F1'] = f1_per_fold
df_scores['Loss'] = loss_per_fold
df_scores['Accuracy'] = acc_per_fold
df_scores['Precision'] = prec_per_fold
df_scores['Recall'] = rec_per_fold

df_scores.to_csv(os.path.abspath(os.path.join('models', dataset, f'LSTM_results_k-Folds.csv')))

Fold: 1

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4801 samples, validate on 666 samples
Epoch 1/2000
4800/4801 [============================>.] - ETA: 0s - loss: 3.6413 - acc: 0.0344
Epoch 00001: val_loss improved from inf to 3.59253, saving model to /home/dorads/NeuralODE/models/PhonemeSpectra/PhonemeSpectra-LSTM_1.hdf5
4801/4801 [==============================] - 10s 2ms/sample - loss: 3.6413 - acc: 0.0344 - val_loss: 3.5925 - val_acc: 0.0240
Epoch 2/2000
4800/4801 [============================>.] - ETA: 0s - loss: 3.5649 - acc: 0.0448
Epoch 00002: val_loss improved from 3.59253 to 3.55273, saving model to /home/dorads/NeuralODE/models/PhonemeSpectra/PhonemeSpectra-LSTM_1.hdf5
4801/4801 [==============================] - 9s 2ms/sample - loss: 3.5648 - acc: 0.0448 - val_loss: 3.5527 - val_acc: 0.0511
Epoch 3/2000
4800/4801 [============================>.] - ETA: 0s - loss: 3.5000 - acc: 0.0546
Epoch 00003: val_loss improved